### Author: Akshay Ijantkar
### Team: Aqua Wizards
### Project: Surfers Bible

# Import Libraries:

In [1]:
import pandas as pd
import numpy as np
import random
import matplotlib as plt
from matplotlib import pyplot
import seaborn as sns; sns.set()
from scipy.stats import norm 
import matplotlib.pyplot as plt
# For Linear regression
from sklearn.linear_model import LinearRegression
# For split given dataset into train and test set.
from sklearn.model_selection import train_test_split
# To verify models using this metrics 
from sklearn.metrics import mean_squared_error, r2_score

import statsmodels.formula.api as smf
from sklearn.linear_model import LinearRegression
from sklearn import metrics
# v
from matplotlib import rcParams
rcParams['figure.figsize'] = 50,50
import pandas_profiling
pd.set_option('display.max_rows', 1500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
from pandas import ExcelWriter
from pandas import ExcelFile

from pygeocoder import Geocoder

import sys
from weather_au import api
from weather_au import summary
from weather import place, observations, uv_index
import time

import json
import requests
from datetime import datetime, timedelta
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split

from sklearn.metrics import r2_score
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_validate

import catboost as ctb
from catboost import CatBoostRegressor, FeaturesData, Pool
from scipy.stats import uniform as sp_randFloat
from scipy.stats import randint as sp_randInt
from scipy.stats import uniform
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics.pairwise import cosine_similarity
from  sklearn.metrics.pairwise import euclidean_distances
from sklearn.metrics.pairwise import manhattan_distances
from sklearn.metrics.pairwise import pairwise_distances
import re
import pprint

import sqlite3
from sqlite3 import Error
%matplotlib inline

C:\Users\aksha\Anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
Using TensorFlow backend.
C:\Users\aksha\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\aksha\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\aksha\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is 

# Create Weather DF:

In [19]:
weather_beach_req_col_lst = [
 'weather_date',
 'beach_address',
 'beach_name',
 'beach_state',
 'beach_country',
 'beach_latitude',
 'beach_longitude',
 'time',
 'summary',
 'icon',
 'precipIntensity',
 'precipProbability',
 'temperature',
 'apparentTemperature',
 'dewPoint',
 'humidity',
 'pressure',
 'windSpeed',
 'windGust',
 'windBearing',
 'cloudCover',
 'uvIndex',
 'visibility',
 'ozone',
                        ]

weather_df = pd.DataFrame(columns = weather_beach_req_col_lst
                         )

# for col in list(weat_dict['hourly']['data'][0].keys()):
#     weather_df[col] = ""
    
# weather_beach_req_col_lst.remove("date")
weather_df  

,weather_date,beach_address,beach_name,beach_state,beach_country,beach_latitude,beach_longitude,time,summary,icon,precipIntensity,precipProbability,temperature,apparentTemperature,dewPoint,humidity,pressure,windSpeed,windGust,windBearing,cloudCover,uvIndex,visibility,ozone


In [20]:
timestamp_epochs = int(time.mktime(time.strptime("2020-05-09", '%Y-%m-%d'))) + 3600  
timestamp_epochs

1588950000

In [21]:
def get_weather_dict_given_date_loc_func(date_str, lat, lon):
    weather_beach_req_col_lst = [
     'weather_date',
     'beach_address',
     'beach_name',
     'beach_state',
     'beach_country',
     'beach_latitude',
     'beach_longitude',
     'time',
     'summary',
     'icon',
     'precipIntensity',
     'precipProbability',
     'temperature',
     'apparentTemperature',
     'dewPoint',
     'humidity',
     'pressure',
     'windSpeed',
     'windGust',
     'windBearing',
     'cloudCover',
     'uvIndex',
     'visibility',
     'ozone',
                            ]
    weather_df = pd.DataFrame(columns = weather_beach_req_col_lst
                             )    

    timestamp_epochs = int(time.mktime(time.strptime(date_str, '%Y-%m-%d'))) + 3600  
    
    API_KEY = "bdbb908566b674b2e5971ac7f61e1bfc"
    
    get_request = ""
    get_request += "https://api.darksky.net/forecast/"
    get_request += API_KEY
    get_request += "/"
    get_request += str(lat)
    get_request += ","
    get_request += str(lon)
    get_request += ","
    get_request += str(timestamp_epochs)

    response_dict = json.loads(requests.get(get_request).text)

    print("get_request = ", get_request)
    print("****************************************************************************************************")
    #     print("response_dict = ",response_dict)

    if len(list(response_dict.keys())) > 2:
        print("Weather API - Response Successfull :)")
        weather_row_dict = {}
        for per_hr_attri_dict in response_dict['hourly']['data']:
            weather_row_dict["weather_date"] = date_str

            for col in weather_beach_req_col_lst:
                weather_row_dict[col] = row[col]

            for hr_attri in per_hr_attri_dict.keys():
                weather_row_dict[hr_attri] = per_hr_attri_dict[hr_attri]  

            weather_row_dict["nearest-station"] = weat_dict['flags']['nearest-station']
            weather_row_dict["sources"] = weat_dict['flags']['sources']
            weather_row_dict["offset"] = weat_dict['offset']

            weather_df = weather_df.append(weather_row_dict, ignore_index=True)
            
        return weather_df
    else:
        print("Weather API - Response Unsuccessfull :(")
        return weather_df

In [22]:
get_weather_dict_given_date_loc_func(date_str = "2020-05-10", 
                                     lat = -33.869, 
                                     lon = 151.209)

get_request =  https://api.darksky.net/forecast/bdbb908566b674b2e5971ac7f61e1bfc/-33.869,151.209,1589036400
****************************************************************************************************
Weather API - Response Successfull :)


NameError: name 'row' is not defined